In [ ]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_column', 500)

ImportError: No module named geopandas

# LCFF

import and clean lcff data

In [ ]:
# import and clean lcff data
lcff = pd.read_csv('lcff_data/lcffsnapshot18an.csv')
lcff.columns = lcff.columns.str.strip()
# strip leading/trailing spaces
lcff.drop(lcff.tail(1).index,inplace=True)
# extracting state totals, removing from df
# Slicing by district level cds code
lcff['County Code'] = lcff['County Code'].astype(str)
lcff['District Code'] = lcff['District Code'].astype(int)
lcff['District Code'] = lcff['District Code'].astype(str)
lcff['School Code'] = lcff['School Code'].astype(int)
lcff['School Code'] = lcff['School Code'].astype(str)
# Changing to strings
lcff['School Code'] = lcff['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# Front fill school codes to == 7
lcff = lcff.loc[lcff['School Code'] == '0000000']
# select districts, district entries do not have a school level cds code
lcff['cds'] = lcff['County Code'] + lcff['District Code'] + lcff['School Code']
# Combining into cds code
col_name='cds'
first_col = lcff.pop(col_name)
lcff.insert(0, col_name, first_col)
# Moving to front 
lcff.drop('County Code', axis=1, inplace=True)
# lcff.drop('District Code', axis=1, inplace=True)
lcff.drop('School Code', axis=1, inplace=True)
lcff = lcff.rename(columns={'Charter Number': 'charter_number',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_percentage',
                     'Base Grant Funding\nTarget, C-5': 'base_grant',
                     'Supplemental Grant Funding\nTarget, D-7': 'supplemental_grant',
                     'Concentration Grant Funding\nTarget, E-8': 'concentration_grant',
                     'Total LCFF Target Entitlement\nTarget, G-1/F-1': 'total_grants',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_count',
                     'Necessary Small Schools Allowance\nTarget, F-1': 'nec_small_schools',
                     'Local Educational Agency' : 'districtname'      
                           })
lcff = lcff[[  'districtname',
               'unduplicated_pupil_count', 
               'base_grant', 
               'supplemental_grant', 
               'concentration_grant', 
               'total_grants', 
               'cds' ]].copy()

In [ ]:
lcff

In [ ]:
new = pd.read_csv('California_School_District_Areas_2018-19.csv')
new = new.rename(columns={'DistrictName' : 'districtname'})

In [ ]:
new

In [ ]:
k = pd.merge(lcff, new, on=['districtname'], how='left')
k['treatment'] = (k.unduplicated_pupil_count.astype(float) >= 0.55) * 1
k

In [ ]:
k.LocaleDistrict.unique()
# is this useful?

In [ ]:
k.drop('AssistStatus', axis=1, inplace=True)
k.drop('UpdateNotes', axis=1, inplace=True)
k.drop('MRcount', axis=1, inplace=True)
k.drop('MRpct', axis=1, inplace=True)
k.drop('PIcount', axis=1, inplace=True)
k.drop('PIpct', axis=1, inplace=True)
k.drop('NRcount', axis=1, inplace=True)
k.drop('NRpct', axis=1, inplace=True)
k.drop('cds', axis=1, inplace=True)
k = k.rename(columns={'DistrictName' : 'districtname',
                  'AAcount': 'african_american',
                  'AApct' : 'african_american_percent',
                  'AIcount' : 'american_indian',
                  'AIpct' : 'american_indian_percent',
                  'AScount' : 'asian',
                  'ASpct' : 'asian_percent',
                  'FIcount' : 'filipino',
                  'FIpct' : 'filipino_percent',
                  'HIcount' : 'hispanic',
                  'HIpct' : 'hispanic_percent',
                  'WHcount' : 'white',
                  'WHpct' : 'white_percent',
                  'ELcount' : 'english_learner',
                  'ELpct' : 'english_learner_percent',
                  'FOScount' : 'foster',
                  'FOSpct' : 'foster_percent',
                  'HOMcount' : 'homeless',
                  'HOMpct' : 'homeless_percent',
                  'MIGcount' : 'migrant',
                  'MIGpct' : 'migrant_percent',
                  'SWDcount' : 'disabilities',
                  'SWDpct' : 'disabilities_percent',
                                                    })

In [ ]:
col_name = 'CountyName'
first_col = k.pop(col_name)
k.insert(0, col_name, first_col)
col_2 = 'EnrollTotal'
second_col = k.pop(col_2)
k.insert(0, col_2, second_col)
# this sucks, is there an easier way to reorder columns?

In [ ]:
k.loc[k['CountyName'] == 'Alameda'].sort_values(['EnrollTotal'], ascending=False)

Alameda's top districts have a large delta between their unduplicated pupil counts.

In Oakland Unified, the majority (76%) of students are categorized as disadvantaged, compared to only 27% at Fremont Unified. 



# Test scores

In [ ]:
mathpop = pd.read_csv("math.csv", index_col=0)
mathpop.columns = mathpop.columns.str.strip()
# strip leading/trailing spaces
engpop = pd.read_csv("ela.csv", index_col=0)
engpop.columns = engpop.columns.str.strip()
# strip leading/trailing spaces
mathpop.shape

In [ ]:
mathpop.head()

In [ ]:
mathpop.loc[mathpop['districtname'] == 'Alameda Unified']
# math scores from the disadvantaged school

In [ ]:
mathpop.loc[mathpop['districtname'] == 'Fremont Unified']
# math scores from the advantaged school

In [ ]:
sample = k.loc[(k['unduplicated_pupil_count'].astype(float) > 0.45) & (k['unduplicated_pupil_count'].astype(float) < 0.65)]
sample.shape

In [ ]:
sample

### Working backwards: what do we want?

In a county: <br>
   - pick districts where UPC > 0.55 <br> 
       - then drop districts where UPC <= 0.65 (this will constitute the treatment group) <br>
   - for each district picked: <br>
       - pair with a district where UPC closest to 0.5499, then closest in EnrollTotal, SEDpct, english_learner_percent, hispanic_percent <br>
     <br>
   - match all selected districts with student groups and grades <br>
   - regress treatment group onto their paired districts
   
Repeat for each county

In [ ]:
t = 100
# target
x = 77
# option 1
y = 120
# option 2
l = [x, y]
# list 

In [ ]:
for target in t:

In [ ]:
given_value = 2
a_list = [1, 5, 8]
absolute_difference_function = lambda list_value : abs(list_value - given_value)

In [ ]:
closest_value = min(a_list, key=absolute_difference_function)

print(closest_value)